In [1]:
%cd /root/share/Real/KAIST/word_chain/alphazero-general/
%load_ext autoreload
%autoreload 2   

/root/share/Real/KAIST/word_chain/alphazero-general


In [2]:
import numpy as np
import pandas as pd
from alphazero.utils import dueum
from joblib import load, dump
from torch_geometric.utils import dense_to_sparse
import torch

In [13]:
def construct_initial_mat():
    """
    두음법칙 고려 없이 순수한 인접행렬 생성
    """
    data = pd.read_csv("alphazero/envs/wordchain/data/20241113_scc.csv")

    head_list = data["앞말"].unique()
    tail_list = data["끝말"].unique()
    ch_list = np.union1d(head_list, tail_list)

    idx2ch = list(ch_list)
    for ch in idx2ch:
        if (dueum(ch) not in idx2ch):
            idx2ch.append(dueum(ch))
            
    ch2idx = dict()
    for idx, ch in enumerate(idx2ch):
        ch2idx[ch] = idx

    adj_mat = np.zeros((len(idx2ch), len(idx2ch)))

    for _, row in data.iterrows():
        head_idx = ch2idx[row['앞말']]
        tail_idx = ch2idx[row['끝말']]
        adj_mat[head_idx][tail_idx] += 1

    #참고
    dueum_dict = {}
    for idx in range(len(idx2ch)):
        didx = ch2idx[dueum(idx2ch[idx])]
        if idx != didx:
            dueum_dict[idx] = didx
    
    

    return ch2idx, idx2ch, dueum_dict, adj_mat

In [14]:
ch2idx, idx2ch, dueum_dict, matrix = construct_initial_mat()
edge_index, edge_weight = dense_to_sparse(torch.Tensor(matrix))

In [18]:
#key를 edge_index (head, tail)로 하고 value를 edge의 순서로 하는 dict 생성
edge_dict = dict()
for i in range(edge_index.shape[1]):
    edge_dict[(edge_index[0][i].item(), edge_index[1][i].item())] = i

In [19]:
dump(edge_dict, "alphazero/envs/wordchain/data/edge_dict.pkl")
dump(matrix, "alphazero/envs/wordchain/data/initial_matrix.pkl")
dump(edge_weight, "alphazero/envs/wordchain/data/initial_edge_weight.pkl")
dump(edge_index, "alphazero/envs/wordchain/data/initial_edge_index.pkl")

['alphazero/envs/wordchain/data/initial_edge_index.pkl']